In [ ]:
import pandas as pd
import psycopg2 as p
import os
import json

In [ ]:
# play by play file structure 
# PlayByPlay\Play By Play\Year\Play By Play\JSON\Week\flattened\files 

# path to entire play by play folder
folder = r'C:\Users\mmgri\Desktop\PlayByPlay'

# creating database connection string
conn = p.connect("host=capstonealaindexing2018.postgres.database.azure.com dbname=mmgtest user=hstandeffer@capstonealaindexing2018 password=Alaindexing!")
cur = conn.cursor()

#declare array for dataframes
dfs = []

# iterate thru PlayByPlay
for playbyplay in os.listdir(folder):

    name = os.fsdecode(playbyplay)
    path = os.path.join(folder, name)

    #iterate thru Play by Play 
    for year in os.listdir(path):

        yearName = os.fsdecode(year)
        yearPath = os.path.join(path, yearName) 
        
        print(yearName)
        
        #iterate thru Year
        for pbp in os.listdir(yearPath):

            pbpName = os.fsdecode(pbp)
            pbpPath = os.path.join(yearPath, pbpName)
            
            #iterate thru Play By Play
            for file in os.listdir(pbpPath):
                    
                fileName = os.fsdecode(file)
                filePath = os.path.join(pbpPath, fileName)
                    
                string = 'JSON'
                    
                if fileName == string:
                                     
                    #iterate thru JSON
                    for week in os.listdir(filePath):

                        #get name and path
                        weekName = os.fsdecode(week)
                        weekPath = os.path.join(filePath, weekName)

                        #iterate thru Week
                        for flat in os.listdir(weekPath):

                            flatName = os.fsdecode(flat)
                            flatPath = os.path.join(weekPath, flatName)

                            #finding flattened folder
                            string = 'flattened'

                            if flatName == string:

                                #iterate thru flattened 
                                for game in os.listdir(flatPath):

                                    gameName = os.fsdecode(game)
                                    gamePath = os.path.join(flatPath, gameName)

                                    # find all json files
                                    if gameName.endswith(".json"): 

                                        # read json file to dataframe
                                        df = pd.read_json(os.path.join(gamePath), orient='columns')
                                    
                                        # add dataframe to list of dataframes
                                        dfs.append(df)
                                        
                                        continue
                                    else:
                                        continue                                
                                continue
                            else:
                                continue
                    continue
                else:
                    continue
                    

In [ ]:
# combine list into one dataframe
df = pd.concat(dfs)

# remove commas from descriptions to avoid erros
df['description'] = df['description'].str.replace('[^\w\s]', '')

# normalize dataframes 
dfGame = df[['awayAbbr', 'awayId', 'awayTeam', 'gameId', 'homeAbbr', 'homeId', 'homeTeam']]
dfDrive = df[['defenseAbbr', 'defenseId', 'defenseTeam', 'driveIndex', 'gameId', 'offenseAbbr', 'offenseId',
             'offenseTeam', 'quarter', 'week', 'year']]
dfPlay = df[['awayScore', 'clock', 'defenseId', 'description', 'distance', 'down', 
             'driveIndex', 'endYardLine', 'homeScore', 'offenseId', 
              'playIndex', 'type', 'yardLine', 'yardsGained', 'week', 'year']]

# remove duplicates from game and drive 
dfGame.drop_duplicates(subset ="gameId", keep = 'first', inplace = True) 
dfDrive.drop_duplicates(subset = ["defenseId", "driveIndex", "offenseId", "week", "year"], keep = 'first', inplace = True)

# array for file names and table names 
fileNames = ['game', 'drive', 'play']

# array for normalized dataframes
dfNormalized = [dfGame, dfDrive, dfPlay]

# loop to add files to postgres database
for df, file in zip(dfNormalized, fileNames):

    print(df.shape)
    print(file)
    
    # writes dataframe to a csv 
    df.to_csv(r'C:\Users\mmgri\Desktop\csv\\' + file + '.csv', sep=',', index=False)
    fileString = r'C:\Users\mmgri\Desktop\csv\\' + file + '.csv'
    
    # loads csv to database 
    with open(fileString, 'r') as f:
        next(f)
        cur = conn.cursor()
        cur.copy_from(f, file, sep=',')
        conn.commit()
        
    print(file + ' completed.')
        
